This script (Version A) is designed to pre-process a single `.mat` v7.3 file that contains all channels. This is optimal for small recordings that will not max out the RAM.

### Configuration

In [1]:
import os
import gc
import numpy as np
import pandas as pd

import mne

from tqdm import tqdm
import datetime
import hdf5storage
import collections as cl

import utils__config

In [2]:
os.chdir(utils__config.working_directory)
os.getcwd()

'g:\\My Drive\\Residency\\Research\\Lab - Damisah\\Project - Sleep'

### Parameters

Subject 01 | February 2

In [3]:
file_path = 'Cache/Subject01/S01_Feb02_macro.mat'
dictionary_path = 'Data/Subject01/S01_dictionary.xlsx'
legui_path = 'Cache/Subject01/S01_electrodes.csv'
save_path = 'Cache/Subject01/S01_Feb02_256hz.fif'
sampling_freq = 256

### Convert MAT to MNE Object

Load .MAT data and format

In [4]:
# Load the .MAT (v7.3) dataset using hdf5storage
data = hdf5storage.loadmat(file_path) # , variable_names = ['foo', 'bar']

# Convert time series data from uV to Volts, which MNE expects
time_series = data['time_series'] * 1e-6

# Merge channel numbers with channel dictionary
# (MNE will not accept numbers as channel names)
ch_names = tuple(data['meta_data']['channel_names'][0][0])
ch_names = pd.DataFrame(ch_names, columns = ['number'])

ch_dictionary = pd.read_excel(dictionary_path)
ch_names = ch_names.merge(ch_dictionary, how = 'inner', on = 'number')

# Create list of channel types to pass to MNE info object
channel_map = {'macro' : 'seeg',
               'scalp' : 'eeg',
               'ecg' : 'ecg',
               'emg' : 'emg',
               'eog' : 'eog',
               'micro' : 'misc',
               'ttl' : 'stim',
               'vitals' : 'bio',
               'empty' : 'misc'}

ch_names = ch_names.assign(ch_types = ch_names.type.map(channel_map))

# Sampling Frequency
sfreq = data['meta_data']['sampling_rate'].astype(np.int64)[0][0][0]

# NSx raw timestamps are formatted YYYY-MM-?-DD-HH-MM-SS-MSMSMS
# (so you do not use the third element in the timestamp; 
#  also, MNE only accepts timestamps in the UTC timezone)
raw_time = data['meta_data']['time_stamp'][0][0][0][0].astype(np.int64)
time_start = datetime.datetime(raw_time[0], raw_time[1], raw_time[3], 
                               raw_time[4], raw_time[5], raw_time[6], 
                               raw_time[7], tzinfo = datetime.timezone.utc)

# Convert tmin/tmax to sample numbers if crop times were specified
if ('tmin' in locals() or 'tmin' in globals()):
    tmin = (tmin - time_start).total_seconds()
    tmax = (tmax - time_start).total_seconds()

Create MNE Raw object

In [5]:
# Create Raw object from numpy array and meta-data
info = mne.create_info(ch_names = ch_names.name.to_list(),
                       sfreq = sfreq,
                       ch_types = ch_names.ch_types.to_list())

raw = mne.io.RawArray(data = time_series,
                      info = info)

# Set the starting timestamp
raw.set_meas_date(time_start)

Creating RawArray with float64 data, n_channels=181, n_times=14402918
    Range : 0 ... 14402917 =      0.000 ...  7201.458 secs
Ready.


<RawArray | 181 x 14402918 (7201.5 s), ~19.42 GB, data loaded>

### Preprocessing

Cropping

In [6]:
if ('tmin' in locals() or 'tmin' in globals()):
    print('Start time:', tmin, '| Stop time:', tmax)
    raw.crop(tmin = tmin, tmax = tmax)

Filter and decimate

In [7]:
# Bandpass Filter
# (Note that the .NS3 files already had
#  an 0.3 - 500 Hz filter applied at 
#  the hardware level)
#raw.filter(l_freq = None, h_freq = 60, n_jobs = -1)

# Notch filter to remove 60 Hz line noise
raw.notch_filter(np.arange(60, sampling_freq/2, 60))

# Downsample via decimation
# (it applies a low-pass filter at half the 
#  desired sampling rate prior to decimation
#  in order to prevent aliasing)
raw.resample(sfreq = sampling_freq)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 13201 samples (6.601 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   50.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   51.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   51.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   52.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed:  2.6min finished


<RawArray | 181 x 1843574 (7201.5 s), ~2.49 GB, data loaded>

Re-reference

In [8]:
# Re-reference macro electrodes to macro-CAR
macro_ref = ch_dictionary[ch_dictionary['type'] == 'macro']['name'].to_list()
raw = raw.set_eeg_reference(ref_channels = macro_ref, ch_type = 'seeg')

# Re-reference scalp electrodes to scalp-CAR
scalp_ref = ch_dictionary[ch_dictionary['type'] == 'scalp']['name'].to_list()
raw = raw.set_eeg_reference(ref_channels = scalp_ref, ch_type = 'eeg')

Applying a custom ('sEEG',) reference.
Applying a custom ('EEG',) reference.


Subset electrodes

In [9]:
# LeGUI channel selection (but keep scalp, eog, and ecg)
print('Original channel count:', len(raw.info.ch_names))
legui_df = pd.read_csv(legui_path)
legui_df = legui_df.loc[(legui_df.status == 'accept') & (legui_df.type == 'macro')]
legui_channels = legui_df.elec_label.to_numpy()
other_channels = ch_dictionary.loc[ch_dictionary['type'].isin(['scalp', 'emg', 'eog']), 'name']
keep_channels_2 = legui_channels.tolist() + other_channels.tolist()
raw = raw.pick_channels(keep_channels_2)
print('Channels after LeGUI selection:', len(raw.ch_names))

Original channel count: 181
Channels after LeGUI selection: 86


Export

In [11]:
#raw.plot()

# Note that MNE saves data to the .FIF file format, 
# which has a maximum size of 2GB. Files larger than
# this are automatically split into numbered files.
# When reading those files back into MNE, you only 
# need to specify the first, unnumbered file name.
# It will automatically look for the numbered splits,
# but those files need to be in the same folder.
raw.save(save_path)

Writing g:\My Drive\Residency\Research\Lab - Damisah\Project - Sleep\Cache\Subject01\S01_Feb02_256hz.fif


C:\Users\Layton\AppData\Local\Temp\ipykernel_20724\1224400030.py:10: RuntimeWarning: This filename (g:\My Drive\Residency\Research\Lab - Damisah\Project - Sleep\Cache\Subject01\S01_Feb02_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(save_path)


Closing g:\My Drive\Residency\Research\Lab - Damisah\Project - Sleep\Cache\Subject01\S01_Feb02_256hz.fif
[done]
